# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

### Import the required libraries

In [245]:
import pandas as pd
import numpy as np
from sklearn import metrics
from xgboost import XGBClassifier, XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, classification_report, confusion_matrix, mean_squared_error, mean_absolute_error,r2_score

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [246]:
df = pd.read_csv("ElectricCarData.csv")

In [247]:
df.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


In [248]:
df = df.drop(columns='Model')

### Encode string variables

In [249]:
df.isnull().sum()

Brand              0
AccelSec           0
TopSpeed_KmH       0
Range_Km           0
Efficiency_WhKm    0
FastCharge_KmH     0
RapidCharge        0
PowerTrain         0
PlugType           0
BodyStyle          0
Segment            0
Seats              0
PriceEuro          0
dtype: int64

In [250]:
# df = df.drop(columns=['Model'],axis=1)

In [251]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    object 
 1   AccelSec         103 non-null    float64
 2   TopSpeed_KmH     103 non-null    int64  
 3   Range_Km         103 non-null    int64  
 4   Efficiency_WhKm  103 non-null    int64  
 5   FastCharge_KmH   103 non-null    object 
 6   RapidCharge      103 non-null    object 
 7   PowerTrain       103 non-null    object 
 8   PlugType         103 non-null    object 
 9   BodyStyle        103 non-null    object 
 10  Segment          103 non-null    object 
 11  Seats            103 non-null    int64  
 12  PriceEuro        103 non-null    int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 10.6+ KB


## Split the dataset for training and testing in ratio 80:20

In [252]:
encoder = OrdinalEncoder()

In [253]:
for column in df.columns:
    # print(df[column][0])
    if type(df[column][0]) == str:
        df[[column]] = encoder.fit_transform(df[[column]])

In [254]:
df.head()

,Brand,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,30.0,4.6,233,450,161,50.0,1.0,0.0,2.0,7.0,3.0,5,55480
1,31.0,10.0,160,270,167,7.0,1.0,2.0,2.0,1.0,2.0,5,30000
2,23.0,4.7,210,400,181,36.0,1.0,0.0,2.0,2.0,3.0,5,56440
3,2.0,6.8,180,360,206,32.0,1.0,2.0,2.0,6.0,3.0,5,68040
4,9.0,9.5,145,170,168,2.0,1.0,2.0,2.0,1.0,1.0,4,32997


In [255]:
X = df.drop(columns='PriceEuro')
Y = df['PriceEuro']
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train = scaler.fit_transform(X_train)
# X_train = scaler.transform(X_train)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [256]:
model1 = DecisionTreeRegressor(max_depth=5,min_samples_split=3,max_features='sqrt')
model1.fit(X_train,Y_train)
y_pred = model1.predict(X_test)

In [257]:
r2_score_1 = r2_score(Y_test,y_pred)
mse_score_1 = mean_squared_error(Y_test,y_pred)
mae_score_1 = mean_absolute_error(Y_test,y_pred)

## Predict the outcomes for X test

## Assess the model performance, by using sklearn metrics for regression

In [258]:
#done

## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [259]:
model2 = XGBRegressor(learning_rate=0.2,max_depth=5,objective='reg:squarederror',n_estimators=50)

In [260]:
model2.fit(X_train,Y_train)
y_pred = model2.predict(X_test)

In [261]:
r2_score_2= r2_score(Y_test,y_pred)
mse_score_2 = mean_squared_error(Y_test,y_pred)
mae_score_2 = mean_absolute_error(Y_test,y_pred)

## Predict the outcomes for X test

In [262]:
print(f"MSE - DTR = {mse_score_1} | XGB = {mse_score_2}")
print(f"R2 score - DTR = {r2_score_1} | XGB = {r2_score_2}")
print(f"MAE - DTR = {mae_score_1} | XGB = {mae_score_2}")

MSE - DTR = 293777238.3813657 | XGB = 115190794.01044355
R2 score - DTR = 0.6295544863843181 | XGB = 0.8547473549842834
MAE - DTR = 11222.0376984127 | XGB = 6861.390438988095


## Assess the model performance, by using sklearn metrics for regression

## Compare the performances of both model for at least three regression metircs

In [263]:
print("DecisionTreeRegressor e podobro od XGBRegressor") if model1_r2score > model2_r2score else print("XGBRegressor e podobro od DecisionTreeRegressor")

XGBRegressor e podobro od DecisionTreeRegressor
